# SCS Analysis Workflow

This notebook demonstrates the Snow Cover to SWE (SCS) analysis workflow using the refactored Snow Drought Index package.

In [ ]:
# Import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import geopandas as gpd
from shapely.geometry import Point
import seaborn as sns

# Import snowdroughtindex package
from snowdroughtindex.analysis import scs_analysis
from snowdroughtindex.core import data_preparation, dataset
from snowdroughtindex.utils import visualization

## Workflow Overview

The workflow uses the following key functions from the `scs_analysis` module:

- `calculate_daily_mean_swe()` for calculating daily mean SWE for a basin
- `filter_points_within_shapefile()` for filtering data points within a basin boundary
- `calculate_basin_mean_precipitation()` for calculating mean precipitation across selected stations
- `merge_swe_precip_data()` for merging SWE and precipitation data
- `calculate_swe_p_ratio()` for calculating and analyzing snow cover to precipitation ratio

## 1. Load Data

First, we need to load the SWE data, precipitation data, and basin shapefile.

In [ ]:
# Define data paths
# Replace these paths with your actual data paths
gap_filled_swe_path = 'path/to/SWE_data.nc'
precip_data_path = 'path/to/precipitation_data.csv'
precip_coordinates_path = 'path/to/precipitation_coordinates.csv'
basin_shapefile_path = 'path/to/basin_shapefile.shp'

# Load SWE data
swe_dataset = xr.open_dataset(gap_filled_swe_path)
print("SWE dataset loaded successfully")
print(swe_dataset)

## 2. Calculate Daily Mean SWE

Calculate the daily mean SWE for the entire basin.

In [ ]:
# Calculate daily mean SWE
daily_mean_swe = scs_analysis.calculate_daily_mean_swe(swe_dataset)
print(daily_mean_swe.head())

## 3. Filter Precipitation Stations Within Basin

Filter precipitation stations that fall within the basin boundary.

In [ ]:
# Load precipitation coordinates
precip_coords = pd.read_csv(precip_coordinates_path)
precip_coords_selected = precip_coords[['subid', 'longitude', 'latitude']]

# Filter points within the basin shapefile
points_within = scs_analysis.filter_points_within_shapefile(
    precip_coords_selected, 
    basin_shapefile_path, 
    station_name="BOW RIVER AT BANFF"  # Replace with your station name
)

print(f"Found {len(points_within)} precipitation stations within the basin")
print(points_within.head())

## 4. Calculate Basin Mean Precipitation

Calculate the mean precipitation across the selected stations within the basin.

In [ ]:
# Load precipitation data
precip_data = pd.read_csv(precip_data_path)

# Get the list of station IDs within the basin
station_ids = points_within['subid'].astype(str).tolist()

# Calculate basin mean precipitation
mean_precip = scs_analysis.calculate_basin_mean_precipitation(precip_data, station_ids)
print(mean_precip.head())

## 5. Merge SWE and Precipitation Data

Merge the daily mean SWE and basin mean precipitation data.

In [ ]:
# Merge SWE and precipitation data
merged_data = scs_analysis.merge_swe_precip_data(daily_mean_swe, mean_precip)
print(merged_data.head())

## 6. Filter Data for Snow Season

Filter the data for the snow season (November to May).

In [ ]:
# Filter for snow season
snow_season_data = scs_analysis.filter_snow_season(
    merged_data, 
    start_month=11, 
    start_day=1, 
    end_month=5, 
    end_day=1
)
print(snow_season_data.head())

## 7. Calculate Seasonal Means

Calculate the mean SWE and precipitation for each snow season.

In [ ]:
# Calculate seasonal means
seasonal_means = scs_analysis.calculate_seasonal_means(snow_season_data)
print(seasonal_means.head())

## 8. Filter Complete Snow Seasons

Filter for complete snow seasons based on SWE threshold and date range.

In [ ]:
# Filter for complete snow seasons
complete_seasons = scs_analysis.filter_complete_seasons(
    merged_data, 
    swe_threshold=15,  # 15mm SWE threshold
    start_month=11, 
    start_day=1, 
    end_month=5, 
    end_day=1
)
print(complete_seasons.head())

## 9. Calculate SWE/P Ratio

Calculate the SWE to precipitation ratio and cumulative precipitation.

In [ ]:
# Calculate SWE/P ratio
ratio_data = scs_analysis.calculate_swe_p_ratio(complete_seasons)
print(ratio_data.head())

## 10. Calculate Seasonal Metrics

Calculate seasonal metrics including max SWE, mean SWE/P ratio, and mean cumulative precipitation.

In [ ]:
# Calculate seasonal metrics
seasonal_metrics = scs_analysis.calculate_seasonal_metrics(ratio_data)
print(seasonal_metrics.head())

## 11. Standardize Metrics for Clustering

Standardize the metrics for clustering analysis.

In [ ]:
# Standardize metrics
standardized_metrics = scs_analysis.standardize_metrics(seasonal_metrics, ratio_max=1.0)
print(standardized_metrics.head())

## 12. Classify Snow Drought Types

Classify snow drought types using K-means clustering.

In [ ]:
# Classify snow drought types
classified_data, cluster_centers, cluster_labels = scs_analysis.classify_snow_drought(
    standardized_metrics, 
    n_clusters=3, 
    random_state=0
)

print("Cluster centers:")
print(cluster_centers)
print("\nCluster labels:")
print(cluster_labels)
print("\nClassified data:")
print(classified_data.head())

## 13. Visualize Results

Visualize the results of the SCS analysis.

In [ ]:
# Plot seasonal SWE vs precipitation
fig1 = scs_analysis.plot_seasonal_swe_precip(seasonal_means)
plt.show()

In [ ]:
# Plot snow drought classification
fig2 = scs_analysis.plot_snow_drought_classification(classified_data)
plt.show()

In [ ]:
# Plot precipitation anomaly time series
fig3 = scs_analysis.plot_drought_time_series(
    classified_data, 
    metric='cum_P_anom', 
    year_range=(1999, 2017)
)
plt.show()

In [ ]:
# Calculate ratio anomaly
classified_data['ratio_anomaly'] = classified_data['SWE_P_ratio'] - classified_data['SWE_P_ratio'].mean()

# Plot SWE/P ratio anomaly time series
fig4 = scs_analysis.plot_drought_time_series(
    classified_data, 
    metric='ratio_anomaly', 
    year_range=(1999, 2017)
)
plt.show()

In [ ]:
# Calculate peak SWE anomaly
classified_data['peak_SWE_anomaly'] = classified_data['SWEmax'] - classified_data['SWEmax'].mean()

# Plot peak SWE anomaly time series
fig5 = scs_analysis.plot_drought_time_series(
    classified_data, 
    metric='peak_SWE_anomaly', 
    year_range=(1999, 2017)
)
plt.show()

## 14. Run Complete SCS Analysis Workflow

Alternatively, you can run the complete SCS analysis workflow using the `run_scs_analysis` function.

In [ ]:
# Run complete SCS analysis workflow
results = scs_analysis.run_scs_analysis(
    daily_mean_swe, 
    precip_data, 
    station_ids, 
    swe_threshold=15, 
    n_clusters=3
)

# Access results
complete_seasons = results['complete_seasons']
seasonal_metrics = results['seasonal_metrics']
classified_data = results['classified_data']
cluster_centers = results['cluster_centers']
cluster_labels = results['cluster_labels']

print("SCS analysis completed successfully!")